In [14]:
import math
import functools

import jax
import jax.numpy as jnp 
from jax import random
from jax.tree_util import tree_map

import flax
import flax.linen as nn

import numpy as np

from general.utils import ModelConfig
from jax_impl.model import *

%reload_ext autoreload
%autoreload 2

In [15]:
key = random.PRNGKey(42)

model_config = ModelConfig(
    vocab_size=1024, 
    context_window=64, 
    model_type='gpt-nano')
model_config

ModelConfig(vocab_size=1024, context_window=64, n_embd=48, n_head=3, n_layer=3, model_type='gpt-nano', attn_pdrop=0.1, recid_pdrop=0.1, embd_pdrop=0.1)

In [16]:
rx, rp, rd, key = random.split(key, 4)
init_rngs = {'params': rp, 'dropout': rd}

x = random.normal(rx, (10, 4, 48))

attn = CausalMultiheadAttention(model_config)
params = attn.init(init_rngs, x)

out = attn.apply(params, x, rngs={'dropout': rd})
out.shape

(10, 4, 48)

In [17]:
blk = EncoderBlock(model_config)
params = blk.init(init_rngs, x)

output = blk.apply(params, x, rngs={'dropout': rd})
output.shape

(10, 4, 48)

In [18]:
idx = random.randint(rx, (10, 4), 0, maxval=model_config.vocab_size, dtype=jnp.integer)

emb = Embedding(model_config)
params = emb.init(init_rngs, idx)

output = emb.apply(params, idx, rngs={'dropout': rd})
output.shape

(10, 4, 48)

In [19]:
gpt = GPT(model_config)
params = gpt.init(init_rngs, idx)

output = gpt.apply(params, idx, rngs={'dropout': rd})
output.shape

(10, 4, 1024)

In [20]:
print(gpt.tabulate(init_rngs, idx))

                                         GPT Summary                                         
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ path                                   ┃ outputs            ┃ params                      ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Inputs                                 │ int32[10,4]        │                             │
├────────────────────────────────────────┼────────────────────┼─────────────────────────────┤
│ Dense_0                                │ float32[10,4,1024] │ bias: float32[1024]         │
│                                        │                    │ kernel: float32[48,1024]    │
│                                        │                    │                             │
│                                        │                    │ 50,176 (200.7 KB)           │
├────────────────────────────────────────┼────────────────────┼─────────────────────────────┤
│ Embedding_0/Dropout_0                  │ float32[10,4,48]   │                             │
├────────────────────────────────────────┼────────────────────┼─────────────────────────────┤
│ Embedding_0/Embed_0                    │ float32[4,48]      │ embedding: float32[64,48]   │
│                                        │                    │                             │
│                                        │                    │ 3,072 (12.3 KB)             │
├────────────────────────────────────────┼────────────────────┼─────────────────────────────┤
│ Embedding_0/Embed_1                    │ float32[10,4,48]   │ embedding: float32[1024,48] │
│                                        │                    │                             │
│                                        │                    │ 49,152 (196.6 KB)           │
├────────────────────────────────────────┼────────────────────┼─────────────────────────────┤
│ Embedding_0                            │ float32[10,4,48]   │                             │
├────────────────────────────────────────┼────────────────────┼─────────────────────────────┤
│ EncoderBlock_0/CausalMultiheadAttenti… │ float32[10,4,144]  │ bias: float32[144]          │
│                                        │                    │ kernel: float32[48,144]     │
│                                        │                    │                             │
│                                        │                    │ 7,056 (28.2 KB)             │
├────────────────────────────────────────┼────────────────────┼─────────────────────────────┤
│ EncoderBlock_0/CausalMultiheadAttenti… │ float32[10,3,4,4]  │                             │
├────────────────────────────────────────┼────────────────────┼─────────────────────────────┤
│ EncoderBlock_0/CausalMultiheadAttenti… │ float32[10,4,48]   │                             │
├────────────────────────────────────────┼────────────────────┼─────────────────────────────┤
│ EncoderBlock_0/Dense_0                 │ float32[10,4,192]  │ bias: float32[192]          │
│                                        │                    │ kernel: float32[48,192]     │
│                                        │                    │                             │
│                                        │                    │ 9,408 (37.6 KB)             │
├────────────────────────────────────────┼────────────────────┼─────────────────────────────┤
│ EncoderBlock_0/Dense_1                 │ float32[10,4,48]   │ bias: float32[48]           │
│                                        │                    │ kernel: float32[192,48]     │
│                                        │                    │                             │
│                                        │                    │ 9,264 (37.1 KB)             │
├────────────────────────────────────────┼────────────────────┼─────────────────────────────┤
│ EncoderBlock_0/Dropout_0               │ float32[1